# Transformer scientific typing

## Problem statement
Time series data comes in many different forms: univariate or multivariate, a single instances or multiple instances. 

Correspondingly, there are many different forms of transformations that can be applied to time series data. 

This is an attempt to organise these transformations into categories with common APIs.

1. Find a taxonomy of time series transformations using scientific types based on input and output types
2. Develop transformer class structure/hierarchies in Python based on scitype taxonomy

In [1]:
from sktime.forecasting.all import *
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator
import typing
from typing import Union, NewType
import pandas as pd
import numpy as np

## Data types

### Primitives
* single or multiple primitives
* primitive types or 1d array 

In [2]:
# single or mulitple primitives
Primitive = Union[np.integer, int, np.float, float, str]
Primitives = np.ndarray

### Tabular
* scikit-learn-like data
* 2d arrays: rows represent instances, columns represent variables/features, cells contain Primitives

In [3]:
Tabular = Union[pd.DataFrame, np.ndarray]  # 2d arrays

### Series
* univariate or multivariate
* 1d or 2d arrays: rows represent time points, columns represent variables

In [4]:
# univariate or multivariate
UnivariateSeries = Union[pd.Series, np.ndarray]
MultivariateSeries = Union[pd.DataFrame, np.ndarray]
Series = Union[UnivariateSeries, MultivariateSeries]

In [5]:
def make_series(n_timepoints=10, n_variables=1):
    assert n_timepoints > 1
    assert n_variables > 0
    index = pd.date_range("2020-01-01", periods=n_timepoints, freq="D")
    if n_variables == 1:
        return pd.Series(np.random.lognormal(size=n_timepoints), index=index, name="Data")
    else:
        return pd.DataFrame(np.random.lognormal(size=(n_timepoints, n_variables)), index=index)

In [6]:
y = make_series(n_timepoints=10)
Y  = make_series(n_timepoints=10, n_variables=3)
y_train, y_test = temporal_train_test_split(make_series(n_timepoints=20))

### Series-as-features
* panel/longitudinal data with multiple instances of Series
* univariate or multivariate
* nested/3d arrays, potentially also awkward array: rows represent instances, columns represent variables/features, 3rd dimension represents time points

In [7]:
SeriesAsFeatures = Union[pd.DataFrame, np.ndarray]

## Transformation types

### 1. Series -> Primitives
* encapsulated in functions - not classes - because not fittable, so no parameters have to be stored (not stateful)
* may have tuneable arguments (e.g. which quantile to extract), which can be exposed in comopsitions via FunctionTransformer
* optionally, wrapped in classes for convenience

#### Examples:
* mean
* quantile

In [8]:
def series_to_primitive(x: Series, **kwargs) -> Primitives:
    # non-fittable
    pass

In [9]:
# example
def quantile(x, q=0.5):
    # non-fittable
    return np.quantile(x, q, axis=0)

In [10]:
quantile(y)

1.7283128500854787

In [11]:
quantile(Y)

array([0.93919498, 0.566029  , 0.79750065])

In [12]:
# turn functions into classes for model composition 
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(quantile)
transformer.fit_transform(y)

1.7283128500854787

#### Aside: Callable transformers
Optionally, we can create callable transformers, but not sure if they add much. 

In [13]:
# alternatively, base class with __call__
class _SeriesToPrimitivesTransformer(BaseEstimator):
    
    def __init__(self, **kwargs):
        self.func = None
        self.kwargs = kwargs
        self._fixed_kwargs = None
    
    def __call__(self, X: Series, q=0.5) -> Primitives:
        # call function with kwargs
        # ignore kwargs passed to __init__
        return self.func(X, q, **self._fixed_kwargs)
    
    def fit(self, X: Series, y=None):
        # empty fit
        return self
    
    def transform(self, X: Series, y=None) -> Primitives:
        return self.func(X, **self.kwargs)
    
    def fit_transform(self, X: Series, y=None) -> Primitives:
        return self.fit(X, y).transform(X, y)
    
    def inverse_transform(self, X: Series, y=None) -> Primitives:
        # any examples of inverse transform? 
        # all series -> primitive transforms seem lossy 
        raise NotImplementedError("abstract method")

In [14]:
# example
class QuantileTransformer(_SeriesToPrimitivesTransformer):
    
    def __init__(self, q=0.5):
        self.q = q
        self._fixed_kwargs = {"axis": 0}
        self.func = np.quantile
        
    def transform(self, X, y=None):
        return self.func(X, self.q, **self._fixed_kwargs)
    
    def fit_transform(self, X: Series, y=None) -> Primitives:
        return self.fit(X, y).transform(X, y)

In [15]:
# call, but not too useful, since we still have to instantiate the class
transformer = QuantileTransformer()
transformer(y, q=0.25)

1.072241053089623

In [16]:
transformer = QuantileTransformer(q=0.25)
transformer.fit_transform(y)

1.072241053089623

### 2. Series -> Series
* may be fittable (e.g. extract AR coefficients from series)
* if not fittable, encapsulated as functions, composable via FunctionTransformer, optionally wrapped in classes for convenience
* if fittable, encapsulated in classes
* input and output of transform may have different number of time points and different index
* input and output may have different domain (no longer on the same time line)
* in a forecasting pipeline, fit_transform is called before call fit of the final forecaster, and inverse_transform is called after calling predict of the final forecaster

#### Examples
* Box-Cox transform
* logarithm
* smoothing, filters
* detrender
* deseasonalisation 
* Change point annotator (e.g. returns sequence of change point waiting times)
* Adding holiday dummies 
* Adding Fourier terms

In [17]:
def series_to_series(x: Series, **kwargs) -> Series:
    # non-fittable
    pass

In [18]:
def fourier_transform(x): 
    return np.fft.fft(x, axis=0)

In [19]:
fourier_transform(y).shape

(10,)

In [20]:
fourier_transform(Y).shape

(10, 3)

In [21]:
# scikit-learn FunctionTransformer
transformer = FunctionTransformer(fourier_transform)
transformer.fit_transform(y).shape

(10,)

In [22]:
# bespoke class implementations
class _SeriesToSeriesTransformer(BaseEstimator):
    
    def fit(self, X: Series, y=None):
        # empty fit by default, but some series-to-series transformers 
        # are fittable (detrender)
        return self
    
    def transform(self, X: Series, y=None) -> Series:
        raise NotImplementedError("abstract method")
        
    def inverse_transform(self, X: Series, y=None) -> Series:
        raise NotImplementedError("abstract method")
    
    def fit_transform(self, X: Series, y=None) -> Series:
        return self.fit(X, y).transform(X, y)

In [23]:
# example
class LogTransformer(_SeriesToSeriesTransformer):
    
    def transform(self, X, y=None):
        return np.log(X)
    
    def inverse_transform(self, X, y=None):
        return np.exp(X)

In [24]:
transformer = LogTransformer()
transformer.fit_transform(Y)

,0,1,2
2020-01-01,-0.601309,-1.270153,-0.527131
2020-01-02,-2.026348,-0.607713,-0.496220
2020-01-03,1.610769,-0.531942,0.019720
2020-01-04,-0.424994,-1.389078,1.048512
2020-01-05,0.902997,2.150501,-1.338515
2020-01-06,0.202628,0.454866,-0.680210
2020-01-07,0.894125,1.227955,-0.013923
2020-01-08,0.230705,-1.164551,0.239025
2020-01-09,-0.793806,0.392698,1.206601
2020-01-10,-0.573338,-0.620723,-1.417919


### Special case: Featurizer
In a forecasting pipeline, fit_transform is called before calling fit of the final forecaster and transform is called *before* calling predict of the final forecaster. So, Featurizers need to be aware of the forecasting horizon and need to be able to return transformed data even when y (forecasted values) are not available yet.

In [25]:
class _SeriesToSeriesFeaturizer(BaseEstimator):
    
    def fit(self, X: Series, y=None):
        # empty fit by default, but some series-to-series transformers 
        # are fittable (detrender)
        return self
    
    def transform(self, X: Series, y=None) -> [Series, Series]:
        raise NotImplementedError("abstract method")
        
    def inverse_transform(self, X: Series, y=None) -> [Series, Series]:
        raise NotImplementedError("abstract method")
    
    def fit_transform(self, X: Series, y=None) -> [Series, Series]:
        return self.fit(X, y).transform(X, y)

In [26]:
# examples
from pmdarima.preprocessing.exog.fourier import _fourier_terms

def _get_index(X=None, y=None):
        assert X is not None or y is not None
        if X is None:
            return y.index
        else:
            return X.index

class HolidayFeaturizer(_SeriesToSeriesTransformer):
    
    def __init__(self, calendar=None):
        self.calendar = calendar
    
    def transform(self, X, y=None):
        index = _get_index(X, y)
        start, end = index[0], index[-1]
        holidays = self.calendar.holidays(start, end, return_name=True)
        if holidays.empty:
            return X
        
        Xt = pd.get_dummies(holidays.reindex(y))
        return pd.concat([X, Xt], axis=1)
    
class FourierFeaturizer(_SeriesToSeriesTransformer):
    
    def __init__(self, sp=None, k=None):
        self.sp = sp
        self.k = k
        
    def fit(self, X, y=None):
        index = _get_index(X, y)
        assert self.sp is not None
        if self.k is None:
            k = self.sp // 2
        else:
            k = self.k
            
        if 2 * k > self.sp or k < 1:
            raise ValueError("k must be a positive integer not greater "
                             "than sp//2")

        # Compute the periods of all Fourier terms. Since R allows multiple
        # seasonality and we do not, we can do this much more simply.
        p = ((np.arange(k) + 1) / self.sp).astype(np.float64)  # 1:K / m

        # If sinpi is 0... maybe blow up?
        # if abs(2 * p - round(2 * p)) < np.finfo(y.dtype).eps:  # min eps

        self.p_ = p
        self.k_ = k
        self.n_ = index.shape[0]
        self._start = index[0]
        self._cutoff = index[-1]
        return self
    
    def transform(self, X, y=None):
        index = _get_index(X, y)
        # extrapolate fourier terms 
        fh = ForecastingHorizon(index, is_relative=False)
        start = self._start
        cutoff = self._cutoff
        times = fh.to_absolute_int(start, cutoff).to_numpy(dtype=np.float64) + 1
        Xt = pd.DataFrame(_fourier_terms(t.p_, times), index=index)
        return pd.concat([X, Xt], axis=1)

In [27]:
# fitting 
X_train = None
t = FourierFeaturizer(sp=12, k=1)
Xt = t.fit_transform(X_train, y=y_train)
Xt.tail(5)

,0,1
2020-01-11,-4.999999e-01,8.660255e-01
2020-01-12,1.748456e-07,1.000000e+00
2020-01-13,5.000002e-01,8.660253e-01
2020-01-14,8.660255e-01,4.999998e-01
2020-01-15,1.000000e+00,-2.185570e-07


In [28]:
# predicting
X_test = None
fh = ForecastingHorizon(y_test.index, is_relative=False)

# pre-initialised empty y_pred with fh as index
if X_test is None:
    X_test = pd.DataFrame(index=fh.to_absolute(t._cutoff).to_pandas(), dtype=np.float)
X_test = t.transform(X_test)
X_test.head(5)

,0,1
2020-01-16,8.660253e-01,-0.500000
2020-01-17,4.999998e-01,-0.866026
2020-01-18,-2.622683e-07,-1.000000
2020-01-19,-5.000002e-01,-0.866025
2020-01-20,-8.660255e-01,-0.500000


### 3. Series-as-features -> Tabular
* usually fittable and encapsualted in classes
* input of fit and transform always needs to have the same number of columns and time points, but may have different number of instances
* input and output of transform always has the same number of instances, but may have different numbers of columns and time points

#### Examples
* Shapelet transform (returns minimum distance of each instance to found shapelets as tabular matrix)
* Feature extractors
* Tabularizer/TimeBinner
* composite transformers, e.g. applying series-to-primitives transform iteratively over instances/rows

In [29]:
class _SeriesAsFeaturesToTabularTransformer:

    def fit(self, X: SeriesAsFeatures, y=None):
        return self
        
    def transform(self, X: SeriesAsFeatures, y=None) -> Tabular:
        return Xt  # tabular

    def inverse_transform(self, X: SeriesAsFeatures, y=None) -> Tabular:
        return Xt  # tabular
    
    def fit_transform(self, X: SeriesAsFeatures, y=None) -> Tabular:
        return self.fit(X, y).transform(X, y)

In [30]:
# example
from sktime.transformers.series_as_features.summarize import RandomIntervalFeatureExtractor
from sktime.utils._testing import make_classification_problem

In [31]:
X, y = make_classification_problem()
X.columns = ["var"]
transformer = RandomIntervalFeatureExtractor()
Xt = transformer.fit_transform(X)
Xt.head()

,var_3_9_mean,var_11_13_mean,var_6_14_mean,var_11_16_mean
0,-0.082192,0.120047,0.042570,-0.013457
1,20.188997,20.038806,20.079160,20.000210
2,0.038663,0.247701,0.144171,0.253148
3,0.122148,0.406688,0.320673,0.219109
4,0.073882,0.080773,0.142635,0.373833


In [32]:
from sklearn.base import clone
# example
class InstanceTransformer(_SeriesAsFeaturesToTabularTransformer):
    
    def __init__(self, transformer):
        self.transformer = transformer
        
    def fit(self, X, y=None):
        assert isinstance(self.transformer, _SeriesToPrimitivesTransformer), "transformer must be a SeriesToPrimitivesTransformer"
        transformer = clone(self.transformer)
        self.transformer_ = transformer.fit(X, y)
        return self
    
    def transform(self, X, y=None):
        assert X.shape[1] == 1, "X must be univariate"
        Xt = np.zeros((X.shape[0], 1))
        for i, (_, x) in enumerate(X.iterrows()):
            x = x.iloc[0]  # unnest
            Xt[i] = self.transformer_.transform(x)
        return Xt

In [33]:
series_to_primitive_transformer = QuantileTransformer()
transformer = InstanceTransformer(series_to_primitive_transformer)
transformer.fit_transform(X)

array([[-6.37745969e-02],
       [ 2.01056542e+01],
       [ 4.75435875e-02],
       [ 7.37238897e-02],
       [-9.09609977e-02],
       [-2.46127900e-01],
       [ 1.02652208e-02],
       [ 1.20824901e-03],
       [-1.16664139e-02],
       [ 2.02438812e+01],
       [ 2.00128580e+01],
       [ 1.99848365e+01],
       [-1.14447991e-01],
       [ 2.01442869e+01],
       [-1.24710635e-01],
       [ 6.70814920e-02],
       [ 2.00023348e+01],
       [ 2.00262489e+01],
       [-1.67622196e-02],
       [ 1.98805785e+01]])

### 4. Series-as-features -> Series-as-features
* usually fittable and encapsualted in classes
* input of fit and transform always needs to have the same number of columns and time points, but may have different number of instances
* input and output of transform always has same number of instances, but may have different numbers of columns and time points (including going from time-homogeneous data to time-heterogenous data)

#### Examples
* dictionary-based transforms 
* time series segmentation, e.g. splitting a time series columns into multiple ones, see IntervalSegmenter
* time series concatenation, e.g. merging multiple time series columns into a single one, see ColumnConcatenator
* composite transformers, e.g. applying series-to-series or series-to-primitive transforms iteratively over instances/rows

In [34]:
class _SeriesAsFeaturesToSeriesAsFeaturesTransformer:

    def fit(self, X: SeriesAsFeatures, y=None):
        return self
        
    def transform(self, X: SeriesAsFeatures, y=None) -> SeriesAsFeatures:
        return Xt  # tabular

    def inverse_transform(self, X: SeriesAsFeatures, y=None) -> SeriesAsFeatures:
        return Xt  # tabular
    
    def fit_transform(self, X: SeriesAsFeatures, y=None) -> SeriesAsFeatures:
        return self.fit(X, y).transform(X, y)

In [35]:
# example
from sktime.transformers.series_as_features.segment import RandomIntervalSegmenter

In [36]:
transformer = RandomIntervalSegmenter(n_intervals=3)
Xt = transformer.fit_transform(X)
Xt.head()

[[14 16]
 [ 3  8]
 [ 2  4]]


,var_14_16,var_3_8,var_2_4
0,"[-0.2647499738541997, -0.07418519836514284]","[0.5599010138863696, -0.7093008204543497, -0.0...","[0.1011900125563006, 0.5599010138863696]"
1,"[19.645199257987077, 19.951407949803734]","[20.90092074650609, 19.73655743270104, 20.2163...","[19.279093432874262, 20.90092074650609]"
2,"[0.05899819153821709, 0.30505610018580925]","[0.03608898341015577, 0.40744981479662834, -0....","[0.8686462135567213, 0.03608898341015577]"
3,"[0.0992092591317817, 0.13471965266355374]","[-0.5828494648509979, -0.23008299207929372, -0...","[0.41137170937860945, -0.5828494648509979]"
4,"[1.1106714887177418, 0.7447159807103513]","[-0.9584539499174366, -0.6811243602822077, 0.9...","[-0.4256495420687192, -0.9584539499174366]"
